# Generate first draft for content

It could help authors, if instead of having to start new content from zero, a Langchain Agent would generate a first draft that they then refine before submitting for review. In this notebook we want to explore how this idea could work.

## Set up tools for agent

First, we have to prepare some tools that we can provide to the agent. We obviously need a large language model (LLM). Additionally, we can equip the agent with already written tools for using a calculator, search engine, Wikipedia or Wolfram Alpha. You can view the full list [here](https://python.langchain.com/docs/integrations/tools/). It is also possible to define own tools.

In [1]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable


### LLM
We choose OpenAI as the LLM provider for no particular reason other than that we yet have to try others and compare.

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file for OPENAI_API_KEY

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

### Tools
LLMs are not good at calculating, so let's provide a calculator for calculation tasks. Generally, LLMs are bad at anything maths. This is unfortunate for serlo.org having a maths focus. We may later consider also providing Wolfram Alpha as a tool to help. We would first have to find out how to get access to that, though.

In [4]:
from langchain.agents import load_tools

In [5]:
tools = load_tools(["llm-math"], llm=llm)

To prevent the LLM from producing so called "hallucinations", we equip it with Wikipedia for fact checking:

In [6]:
!pip install wikipedia

Defaulting to user installation because normal site-packages is not writeable


In [7]:
tools += load_tools(["wikipedia"])

If we use a search engine, we may have to consider copyright of found results.

In [8]:
from langchain.tools import BraveSearch
search_tool = BraveSearch.from_api_key(api_key="BSAv1neIuQOsxqOyy0sEe_ie2zD_n_V", search_kwargs={"count": 3})

#test it
#print(search_tool.run("schulferien bayern"))

#tools += search_tool

### Advanced tools
We would like our generated article to not only be text but also have some helpful graphics. An option to consider for this, is using a Python agent as a tool that would create the Python code for plotting functions or diagrams and run it. 

In [9]:
# todo

## Set up agent for content creation

After defining our tools, we now create the agent to use them:

In [10]:
from langchain.agents import initialize_agent, AgentType

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True) # for this experimentation, we of course want to know what the agent does

## Prepare article creation task for the agent

We start by creating a few variables that could be the information retrieved from the author in an actual product. We could try different inputs here.

In [11]:
title: str = "Ableitung"
jahrgangsstufe: int = 8
schulart: str = "Hauptschule"
schulfach: str = "Mathematik"

In [12]:
from langchain.prompts import ChatPromptTemplate

style = """Deutsch \
in einfacher Sprache, so dass selbst lernschwache %d-Klässler der Schulart %s das Thema gut verstehen
""" % (jahrgangsstufe, schulart)

template_string = """
Erstelle einen Schulbuchartikel im Fach {schulfach} zum Thema {title} im folgenden Stil: {style}. \
Verwende LateX für mathematische Notation. \
Für Grafiken zur Veranschaulichung kannst du 
"""
prompt_template = ChatPromptTemplate.from_template(template_string)
article_creation_task = prompt_template.format_messages(style=style, title=title, schulfach=schulfach)[0].content

## Let the agent perform article creation task

Let's see how our agent does with the given task:

In [13]:
#agent.run(article_creation_task)

## Get article into Serlo Editor format

To display the article in the editor, we need it in the Serlo Editor JSON format described in the [documentation wiki](https://github.com/serlo/documentation/wiki/Content-format). So we would make a text plugin per text paragraph, equations plugin per LateX expression, and so on. This seems to be a task to be solved programatically, not with machine learning models.

In [14]:
# todo

## Prepare solution creation task for the agent

Since creating a solution is in a lot of ways not so different from creating an article, we now try this as well. First we need to get an exercise:

In [15]:
import json
import requests

def get_exercise_from_uuid(uuid_article):
    req = requests.post(
        #"https://de.serlo.org/api/frontend/localhost-graphql-fetch",
        "https://api.serlo-staging.dev/graphql",
        headers = {
            "Content-Type": "application/json",
        },
        json = {
            "query": """
            query {
              uuid(id: %d) {
                ... on Exercise {
                  currentRevision { 
                    id
                    content
                  }
                }
              }
            }
            """ % uuid_article
        }
    )
    
    return req.json()